In [1]:
import os
import sys

In [2]:
WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/trabalho_final/"
PYSERINI_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/"

TREC_EVAL_FULLPATH=PYSERINI_FOLDER+"tools/eval/trec_eval.9.0.4/trec_eval"

os.environ["ANSERINI_CLASSPATH"]="/media/eduseiti/bigdata01/unicamp/ia368v_dd/anserini/target"

In [3]:
os.chdir(WORKING_FOLDER)

In [4]:
from common_tools import *

In [5]:
from pyserini.search.lucene import LuceneSearcher
from pyserini.analysis import get_lucene_analyzer

In [6]:
import glob

In [7]:
EVALUATION_QUESTIONS_FILE="Questões - Dataset Unicamp IR - Página1.tsv"

In [8]:
BM25_INDEX_FOLDER="bm25/windowed_v2_index"
BM25_RETRIEVAL_FOLDER="BM25_retrieval"
BM25_RETRIVED_DOCS_RESULT_FILENAME_FORMAT="query_id_{:03}.tsv"

BM25_QUERY_EVALUATION_FOLDER="BM25_evaluation"
BM25_QUERY_EVALUATION_RESULT_FILENAME_FORMAT="{}_{}_evaluation_{:04}_{:04}.tsv"

### Prepare to execute the BM25 search and store the results

In [ ]:
eval_queries_df = pd.read_csv(EVALUATION_QUESTIONS_FILE, sep="\t")

In [ ]:
eval_queries_df

In [ ]:
BM25_searcher = LuceneSearcher(BM25_INDEX_FOLDER)
analyzer = get_lucene_analyzer("pt")

BM25_searcher.set_analyzer(analyzer)

In [ ]:
if not os.path.exists(BM25_RETRIEVAL_FOLDER):
    os.makedirs(BM25_RETRIEVAL_FOLDER)

### Loop through the questions and retrieve the BM25 associated documents

In [ ]:
for i, question in eval_queries_df.iterrows():
    
    print("Handling query {}: {}...".format(i, question['Pergunta']))
    
    document_hits = BM25_searcher.search(question['Pergunta'], k=1000)
    
    with open(os.path.join(BM25_RETRIEVAL_FOLDER, BM25_RETRIVED_DOCS_RESULT_FILENAME_FORMAT.format(i)), "w") as output_file:
        for j, document in enumerate(document_hits):
            document_dict = json.loads(document.raw)
            
            if j > 0:
                output_file.write("\n")
            
            output_file.write("{}\t{}\t{}\t{}\t{}".format(i,
                                                          question['Pergunta'],
                                                          document_dict['id'],
                                                          document.score,
                                                          re.sub('[\r\n\t]', ' ', document_dict['contents'])))

### Now, evaluate the retrieved documents relevance

In [9]:
initialize_openai()

In [10]:
if not os.path.exists(BM25_QUERY_EVALUATION_FOLDER):
    os.makedirs(BM25_QUERY_EVALUATION_FOLDER)

In [11]:
def relevance_evaluation(retrieval_folder, starting_position=0, ending_position=10, which_query=None, model=MODEL_GPT3):
    if which_query is None:
        which_queries = glob.glob(os.path.join(retrieval_folder, "query_id_*.tsv"))
    else:
        which_queries = glob.glob(os.path.join(retrieval_folder, "query_id_{:03}.tsv".format(which_query)))
    
      
    for query in which_queries:
        
        print("Evaluating query results interval [{}, {}] for file {}...\n".format(starting_position, ending_position, query))
        
        query_docs_df = pd.read_csv(query, sep="\t", header=None, names=['query_id', 'query_text', 'doc_id', 'doc_score', 'doc_text'])
        
        relevance_results = []
        
        for i, query_doc in query_docs_df.iloc[starting_position:ending_position].iterrows():
            relevance_results.append(execute_LLM_passage_relevance_evaluation(query_doc['query_text'], query_doc['doc_text'], model=model))
            
        
        results_df = pd.concat([query_docs_df.iloc[starting_position:ending_position], 
                                pd.DataFrame(list(range(starting_position, ending_position)), columns=['doc_position']),
                                pd.DataFrame(relevance_results)], axis=1)
        results_df.to_csv(os.path.join(BM25_QUERY_EVALUATION_FOLDER, BM25_QUERY_EVALUATION_RESULT_FILENAME_FORMAT.format(os.path.splitext(os.path.basename(query))[0], 
                                                                                                                         model,
                                                                                                                         starting_position, 
                                                                                                                         ending_position)), 
                          index=None, sep="\t")
        
        print("Total cost: {}; total duration: {}".format(results_df['cost'].sum(), results_df['duration'].sum()))

In [ ]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, which_query=0)

In [ ]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, which_query=10, model=MODEL_DAVINCI3)

In [ ]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, which_query=10)

In [ ]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, which_query=50)

In [ ]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, which_query=73)

In [14]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, starting_position=10, ending_position=20, which_query=73)

Evaluating query results interval [10, 20] for file BM25_retrieval/query_id_073.tsv...

++++++++++++++++++++++++++
Passagem: "Embora seja um país de colonização portuguesa, outros grupos étnicos deixaram influências profundas na cultura nacional, destacando-se os povos indígenas, os africanos, os italianos e os alemães. As influências indígenas e africanas deixaram marcas no âmbito da música, da culinária, do folclore, do artesanato, dos caracteres emocionais e das festas populares do Brasil, assim como centenas de empréstimos à língua portuguesa. É evidente que algumas regiões receberam maior contribuição desses povos: os estados do Norte têm forte influência das culturas indígenas, enquanto algumas regiões do Nordeste têm uma cultura bastante africanizada, sendo que, em outras, principalmente no sertão, há uma intensa e antiga mescla de caracteres lusitanos e indígenas, com menor participação africana."
Pergunta: "Qual a principal influência na culinária da região nordeste do brasil?



Pontuação: 9; Razão: a passagem responde claramente que a principal influência na culinária da região nordeste do Brasil é a africana. Além disso, a passagem apresenta informações adicionais sobre outras influências culturais em diferentes regiões do país.

LLM document aggregation duration: 2.6384541988372803


++++++++++++++++++++++++++
Passagem: "Conteúdo e Metodologia da História  39- Sobre a Cultura Afro-Brasileira, marque a opção incorreta:  a) O Brasil tem a maior população de origem africana fora da África e, por isso, a  cultura desse continente exerce grande influência, principalmente, na região Nordeste  do Brasil. b) Hoje, a cultura afro-brasileira é resultado também das influências dos portugueses  e indígenas, que se manifestam na música, religião e culinária. c) Devido à quantidade de escravos recebidos e também pela migração interna  destes, os estados de Maranhão, Pernambuco, Alagoas, Bahia, Minas Gerais, Espírito  Santo, Rio de Janeiro, São Paulo e Rio Grande do Sul



Pontuação: 10; Razão: a passagem responde de forma clara e direta que a principal influência na culinária da região nordeste do Brasil é a cultura africana, que exerce grande influência na região devido à maior população de origem africana fora da África estar no Brasil.

LLM document aggregation duration: 2.5585827827453613


++++++++++++++++++++++++++
Passagem: "Acarajé: prato típico da culinária nordestina, principalmente no estado da Bahia. Ele é um bolinho feito com massa de feijão, recheado com camarões, frito no óleo de dendê. atualizado em 15/10/2020 Você também pode gostar de: Culinária da Região Nordeste Temas Relacionados • Região Nordeste do Brasil Bibliografia Indicada Culiária Nordestina Autor: Vários autores Editora: Senac São Paulo A cozinha africana da Bahia Autor: Radel, Guilherme Editora: Guilherme Radel O Brasil bem temperado - Nordeste Autor: Mendonça, Ana Cecilia N. M. Xavier de Editora: Global Temas: Gastronomia"
Pergunta: "Qual a principal influência na culiná



Pontuação: 10; Razão: a passagem responde de forma clara e direta que a culinária africana é a principal influência na culinária da região nordeste do Brasil. A passagem também fornece informações adicionais sobre a culinária afro-brasileira.

LLM document aggregation duration: 2.404022216796875


++++++++++++++++++++++++++
Passagem: "A influência indígena é também forte no folclore do interior brasileiro, povoado de seres fantásticos como o curupira, o saci-pererê, o boitatá e a iara, entre outros. Na culinária brasileira, a mandioca, a erva-mate, o açaí, a jabuticaba, inúmeros pescados e outros frutos da terra, além de pratos como os pirões, entraram na alimentação brasileira por influência da culinária indígena no Brasil. Essa influência se faz mais forte em certas regiões do país, em que esses grupos conseguiram se manter mais distantes da ação colonizadora, principalmente em porções da Região Norte do Brasil ."
Pergunta: "Qual a principal influência na culinária da região nordes

In [ ]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, which_query=73, model=MODEL_DAVINCI3)

In [12]:
relevance_evaluation(BM25_RETRIEVAL_FOLDER, starting_position=10, ending_position=20, which_query=73, model=MODEL_DAVINCI3)

Evaluating query results interval [10, 20] for file BM25_retrieval/query_id_073.tsv...

++++++++++++++++++++++++++
Passagem: "Embora seja um país de colonização portuguesa, outros grupos étnicos deixaram influências profundas na cultura nacional, destacando-se os povos indígenas, os africanos, os italianos e os alemães. As influências indígenas e africanas deixaram marcas no âmbito da música, da culinária, do folclore, do artesanato, dos caracteres emocionais e das festas populares do Brasil, assim como centenas de empréstimos à língua portuguesa. É evidente que algumas regiões receberam maior contribuição desses povos: os estados do Norte têm forte influência das culturas indígenas, enquanto algumas regiões do Nordeste têm uma cultura bastante africanizada, sendo que, em outras, principalmente no sertão, há uma intensa e antiga mescla de caracteres lusitanos e indígenas, com menor participação africana."
Pergunta: "Qual a principal influência na culinária da região nordeste do brasil?





Pontuação: 2; Razão: não responde a pergunta de forma clara, pois apenas indica indiretamente que a região Nordeste tem uma cultura bastante africanizada, mas não explicita qual é a principal influência na culinária. O assunto da passagem é sobre a diversidade cultural do Brasil, mas não é diretamente o assunto da pergunta.

LLM document aggregation duration: 6.469574451446533


++++++++++++++++++++++++++
Passagem: "Conteúdo e Metodologia da História  39- Sobre a Cultura Afro-Brasileira, marque a opção incorreta:  a) O Brasil tem a maior população de origem africana fora da África e, por isso, a  cultura desse continente exerce grande influência, principalmente, na região Nordeste  do Brasil. b) Hoje, a cultura afro-brasileira é resultado também das influências dos portugueses  e indígenas, que se manifestam na música, religião e culinária. c) Devido à quantidade de escravos recebidos e também pela migração interna  destes, os estados de Maranhão, Pernambuco, Alagoas, Bahia, Minas 





Pontuação: 2; Razão: não responde a pergunta de forma clara, pois apenas indica indiretamente que a culinária da região é marcante e original, mas não explicita qual é a principal influência. O assunto da passagem é sobre temperos típicos da culinária nordestina, mas não é diretamente o assunto da pergunta.

LLM document aggregation duration: 6.752074956893921


++++++++++++++++++++++++++
Passagem: "Os currículos da educação infantil, do ensino fundamental e do ensino médio devem ter base nacional comum, a ser complementada, em cada sistema de ensino e em cada estabelecimento escolar, por uma parte diversificada, exigida pela característica regionais e locais da sociedade da cultura da economia e dos educandos…. exibir mais Hernán Cortés 1238 palavras | 5 páginas xxxxxxxxxx- xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx- xxxxxxxxxxxxxxxxxxxxO Brasil tem a maior população de origem africana fora da África e, por isso, a cultura desse continente exerce grande influênci





Pontuação: 2; Razão: a passagem não responde a pergunta de forma clara, pois apenas indica que a culinária nordestina é influenciada pela culinária africana, mas não explicita qual é a principal influência. O assunto da passagem é sobre o acarajé, que, embora relacionado, não é diretamente o assunto da pergunta.

LLM document aggregation duration: 5.209094285964966


++++++++++++++++++++++++++
Passagem: "Desta forma, este trabalho teve como objetivo relatar as influências dos povos que formaram o Brasil na gastronomia, com foco específico sobre a região Nordeste. O artigo foi realizado através de pesquisa bibliográfica, onde foram consultados livros, artigos e sites na internet que tratavam do presente assunto. Palavras-chave : Culinária do Nordeste."
Pergunta: "Qual a principal influência na culinária da região nordeste do brasil?"
++++++++++++++++++++++++++


Você avalia se uma passagem de texto responde a uma pergunta, indicando uma pontuação de 0 à 10, onde 0 indica que a passa





Pontuação: 10; Razão: a passagem responde à pergunta de forma clara e direta, indicando que a principal influência na culinária da região nordeste do Brasil é a culinária africana.

LLM document aggregation duration: 4.332464694976807


++++++++++++++++++++++++++
Passagem: "A influência indígena é também forte no folclore do interior brasileiro, povoado de seres fantásticos como o curupira, o saci-pererê, o boitatá e a iara, entre outros. Na culinária brasileira, a mandioca, a erva-mate, o açaí, a jabuticaba, inúmeros pescados e outros frutos da terra, além de pratos como os pirões, entraram na alimentação brasileira por influência da culinária indígena no Brasil. Essa influência se faz mais forte em certas regiões do país, em que esses grupos conseguiram se manter mais distantes da ação colonizadora, principalmente em porções da Região Norte do Brasil ."
Pergunta: "Qual a principal influência na culinária da região nordeste do brasil?"
++++++++++++++++++++++++++


Você avalia se u





Pontuação: 8; Razão: a passagem não responde diretamente à pergunta, mas indica que a influência indígena é forte na culinária brasileira, e que essa influência é ainda mais forte em certas regiões do país, como a Região Norte. Embora a passagem não mencione diretamente a Região Nordeste, a informação é suficiente para responder à pergunta.

LLM document aggregation duration: 9.35351276397705


Total cost: 0.26268; total duration: 62.5956757068634
